In [1]:
from keras.applications import vgg16
img_rows, img_cols = 224, 224 
# Re-loads the MobileNet model without the top or FC layers
model=vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


In [2]:
model.layers

In [3]:
for layer in model.layers:
    layer.trainable = False

In [4]:
model.layers[10].trainable

False

In [5]:
from keras.optimizers import RMSprop
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
num_classes = 1
    
top_model = model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(1800,activation='relu')(top_model)
top_model = Dense(1200,activation='relu')(top_model)
top_model = Dense(num_classes,activation='sigmoid')(top_model)
facemodel=Model(inputs=model.input, outputs =top_model)

In [6]:
facemodel.layers


In [7]:
facemodel.compile(optimizer=RMSprop(),loss='binary_crossentropy',metrics=['accuracy'])
facemodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
from keras_preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_dataset = train_datagen.flow_from_directory(
        'Facedataset/train/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')
test_dataset = test_datagen.flow_from_directory(
        'Facedataset/test/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

Found 199 images belonging to 2 classes.
Found 284 images belonging to 2 classes.


In [9]:

facemodel.fit(
        train_dataset,
        steps_per_epoch=199,
        epochs=4,
        validation_data=test_dataset,
validation_steps=200)

Epoch 1/4
199/199 [==============================] - 3162s 16s/step - loss: 0.3234 - accuracy: 0.8758 - val_loss: 2.9443 - val_accuracy: 0.6095
Epoch 2/4
199/199 [==============================] - 3330s 17s/step - loss: 0.1101 - accuracy: 0.9705 - val_loss: 3.5334 - val_accuracy: 0.6298
Epoch 3/4
199/199 [==============================] - 3879s 19s/step - loss: 0.0921 - accuracy: 0.9806 - val_loss: 3.4099 - val_accuracy: 0.6305
Epoch 4/4
  5/199 [..............................] - ETA: 48:51 - loss: 0.0052 - accuracy: 1.0000

C:\Users\Home\anaconda3\envs\mynewenv\lib\site-packages\keras\utils\data_utils.py:616: UserWarning: The input 3 could not be retrieved. It could be because a worker has died.
  UserWarning)


199/199 [==============================] - 3669s 18s/step - loss: 0.0644 - accuracy: 0.9895 - val_loss: 3.2370 - val_accuracy: 0.6090


In [1]:
from keras.preprocessing import image

test_image=image.load_img('Facedataset/test/Akshay goyal/10.jpg',target_size=(224,224))
import numpy as np
test_image1=image.img_to_array(test_image)
test_image1.shape
test_image2=np.expand_dims(test_image1,axis=0)
result=facemodel.predict(test_image2)

Using TensorFlow backend.


NameError: name 'facemodel' is not defined

In [ ]:

import cv2
test_image1 = cv2.imread('Facedataset/test/Akshay goyal/10.jpg')
test_image1=cv2.resize(test_image1, (350,350), interpolation = cv2.INTER_AREA)
font = cv2.FONT_HERSHEY_SIMPLEX

if result[0][0]==1:
    cv2.putText(test_image1,'Hritick Goyal',(10,50), font, 1,(255,255,255),2)
#Display the image
    cv2.imshow("img",test_image1)
    cv2.waitKey(0) 
else:
    cv2.putText(test_image1,'Akshay Goyal',(10,50), font, 1,(255,255,255),2)
#Display the image
    cv2.imshow("img",test_image1)
    cv2.waitKey(0)


In [12]:
train_dataset.class_indices

{'Akshay Goyal': 0, 'Hritick goyal': 1}